In [7]:
# The NLTK library is required:
#!conda install nltk
# We will also use TextBlob:
#!pip install textblob
# Furthermore, in order to use TextBlob(bunchOfText).sentences, I
#   needed to issue the following command:
#!python -m textblob.download_corpora  
#   -- if you need this, you will get an error message telling you so
import random
import logging
import os

os.environ['NLTK_DATA'] = os.getcwd() + '/nltk_data'

from textblob import TextBlob
from config import FILTER_WORDS

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

## Preprocessing
The user is going to input a sentence. It might be messy, like "it mIGht     be MESSY."
We want to first preprocess the text so that BillyBot can understand it best!

Fortunately, NLTK and TextBlob can do the heavy-lifting for us!

In [13]:
def process_input_text(input_text):
    """
    Removes extraneous white space and returns TextBlob object
    """
    return TextBlob(' '.join(input_text.split()))

In [26]:
# Example processing step
processed_text = process_input_text('This is   a  sentence. I made a great         example!')
print(processed_text.sentences)

[Sentence("This is a sentence."), Sentence("I made a great example!")]


## Components of  Sentence

In [ ]:
def find_candidate_parts_of_speech(parsed):
    """
    Given a parsed input (TextBlob object), find the best pronoun, direct noun, adjective, 
    and verb to match their input.
    Returns a tuple of pronoun, noun, adjective, verb any of which may be None if there was 
    no good match
    """
    pronoun = None
    noun = None
    adjective = None
    verb = None
    for sent in parsed.sentences:
        pronoun = find_pronoun(sent)
        noun = find_noun(sent)
        adjective = find_adjective(sent)
        verb = find_verb(sent)
    logger.info("Pronoun=%s, noun=%s, adjective=%s, verb=%s", pronoun, noun, adjective, verb)
    return pronoun, noun, adjective, verb

def find_pronoun(sent):
    """
    Given a sentence (TextBlob Sentence object), find a preferred pronoun to respond with. 
    Returns None if no candidate pronoun is found in the input
    """
    pronoun = None

    for word, part_of_speech in sent.pos_tags:
        # Disambiguate pronouns
        if part_of_speech == 'PRP' and word.lower() == 'you':
            pronoun = 'I'
        elif part_of_speech == 'PRP' and word == 'I':
            # If the user mentioned themselves, then they will definitely be the pronoun
            pronoun = 'You'
    return pronoun

def find_noun(sent):
    """
    Given a sentence (TextBlob Sentence object), find the best candidate noun.
    """
    noun = None

    if not noun:
        for w, p in sent.pos_tags:
            if p == 'NN':  # This is a noun
                noun = w
                break
    if noun:
        logger.info("Found noun: %s", noun)

    return noun

def find_adjective(sent):
    """
    Given a sentence (TextBlob Sentence object), find the best candidate adjective.
    """
    adj = None
    for w, p in sent.pos_tags:
        if p == 'JJ':  # This is an adjective
            adj = w
            break
    return adj

def find_verb(sent):
    """
    Pick a candidate verb for the sentence.
    """
    verb = None
    pos = None
    for word, part_of_speech in sent.pos_tags:
        if part_of_speech.startswith('VB'):  # This is a verb
            verb = word
            pos = part_of_speech
            break
    return verb, pos



In [30]:
# Examples
sentences = processed_text.sentences
print('sentences[0]:',sentences[0])
print('sentences[1]:',sentences[1])

# There should be no verb in the first sentence, but we should
#   be able to isolate 'made' in the second sentence.
find_verb()

sentences[0]: This is a sentence.
sentences[1]: I made a great example!


In [8]:
def starts_with_vowel(word):
    """Check for pronoun compability -- 'a' vs. 'an'"""
    return True if word[0] in 'aeiou' else False

In [11]:
t.pos_tags

[('the', 'DT'),
 ('end', 'NN'),
 ('is', 'VBZ'),
 ('nigh', 'JJ'),
 ('oh', 'UH'),
 ('yea', 'NN')]

## Greetings
The simplest way to approach a greeting is rules-based: make a list of likely greetings and
a list of potential responses.  Since this is a stateless chatbot, it won't get annoyed if you
say "hi" 10,000 times! It picks its response at random.  In practice, the tutorial advises
to cycle through the responses, or at the least hedge against immediate repeats.

In [2]:
# Sentences we'll respond with if the user greeted us
GREETING_KEYWORDS = ("hello", "hi", "greetings", "sup", "what's up",)

GREETING_RESPONSES = ["What do ya want?", "Oh, hey.", "Let's make this quick. I only got about two humps left in me.",
                     "You must want something.", "Hey, I was just about to go get some coffee."]

def check_for_greeting(sentence):
    """If any of the words in the user's input was a greeting, return a greeting response"""
    for word in sentence.words:
        if word.lower() in GREETING_KEYWORDS:
            return random.choice(GREETING_RESPONSES)


# Sentences we'll respond with if we have no idea what the user just said
NONE_RESPONSES = [
    "uh whatever",
    "meet me at the foosball table, bro?",
    "code hard bro",
    "want to bro down and crush code?",
    "I'd like to add you to my professional network on LinkedIn",
    "Have you closed your seed round, dog?",
]
#

## ....

In [3]:
check_for_greeting('hi there')

AttributeError: 'str' object has no attribute 'words'